In [17]:
import numpy as np
import random
import pandas as pd

import datetime
import math, time
import itertools
import datetime
from operator import itemgetter
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler
from math import sqrt
import torch
import torch.nn as nn
from torch.autograd import Variable

import matplotlib.pyplot as plt
import os

In [18]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [19]:
dpath = "/content/drive/My Drive/LAB/data/"
files = os.listdir(dpath)
files.sort()
print(files)

['ADANIPORTS.csv', 'ASIANPAINT.csv', 'AXISBANK.csv', 'BAJAJ-AUTO.csv', 'BAJAJFINSV.csv', 'BAJFINANCE.csv', 'BHARTIARTL.csv', 'BPCL.csv', 'BRITANNIA.csv', 'CIPLA.csv', 'COALINDIA.csv', 'DRREDDY.csv', 'EICHERMOT.csv', 'GAIL.csv', 'GRASIM.csv', 'HCLTECH.csv', 'HDFC.csv', 'HDFCBANK.csv', 'HEROMOTOCO.csv', 'HINDALCO.csv', 'HINDUNILVR.csv', 'ICICIBANK.csv', 'INDUSINDBK.csv', 'INFY.csv', 'IOC.csv', 'ITC.csv', 'JSWSTEEL.csv', 'KOTAKBANK.csv', 'LT.csv', 'MARUTI.csv', 'MM.csv', 'NESTLEIND.csv', 'NTPC.csv', 'ONGC.csv', 'POWERGRID.csv', 'RELIANCE.csv', 'SBIN.csv', 'SHREECEM.csv', 'SUNPHARMA.csv', 'TATAMOTORS.csv', 'TATASTEEL.csv', 'TCS.csv', 'TECHM.csv', 'TITAN.csv', 'ULTRACEMCO.csv', 'UPL.csv', 'VEDL.csv', 'WIPRO.csv', 'ZEEL.csv']


In [20]:
df = pd.read_csv(dpath + files[0]).set_index('Date')
df = df[['Close']]

In [21]:
scaler = MinMaxScaler(feature_range=(-1, 1))
df['Close'] = scaler.fit_transform(df['Close'].values.reshape(-1,1))

In [22]:
def load_data(stock, look_back):
    data_raw = stock.values # convert to numpy array
    data = []

    # create all possible sequences of length look_back
    for index in range(len(data_raw) - look_back):
        data.append(data_raw[index: index + look_back])

    data = np.array(data);
    test_set_size = int(np.round(0.2*data.shape[0]));
    train_set_size = data.shape[0] - (test_set_size);

    x_train = data[:train_set_size,:-1,:]
    y_train = data[:train_set_size,-1,:]

    x_test = data[train_set_size:,:-1]
    y_test = data[train_set_size:,-1,:]

    return [x_train, y_train, x_test, y_test]

look_back = 30 # choose sequence length
x_train, y_train, x_test, y_test = load_data(df, look_back)
print('x_train.shape = ',x_train.shape)
print('y_train.shape = ',y_train.shape)
print('x_test.shape = ',x_test.shape)
print('y_test.shape = ',y_test.shape)

x_train.shape =  (2634, 29, 1)
y_train.shape =  (2634, 1)
x_test.shape =  (658, 29, 1)
y_test.shape =  (658, 1)


In [23]:
x_train = torch.from_numpy(x_train).type(torch.Tensor)
x_test = torch.from_numpy(x_test).type(torch.Tensor)
y_train = torch.from_numpy(y_train).type(torch.Tensor)
y_test = torch.from_numpy(y_test).type(torch.Tensor)

In [27]:
input_dim = 1
hidden_dim = 30
num_layers = 5
output_dim = 1

class RNN(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers, output_dim):
        super(RNN, self).__init__()
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers

        self.rnn = nn.RNN(input_dim, hidden_dim, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_dim).requires_grad_()
        out, (hn) = self.rnn(x, (h0.detach()))
        out = self.fc(out[:, -1, :])
        return out


model = RNN(input_dim=input_dim, hidden_dim=hidden_dim, output_dim=output_dim, num_layers=num_layers)
# summary(model, (look_back, 1))

loss_fn = torch.nn.MSELoss()

optimiser = torch.optim.Adam(model.parameters(), lr=0.01)
print(model)
print(len(list(model.parameters())))
for i in range(len(list(model.parameters()))):
    print(list(model.parameters())[i].size())

RNN(
  (rnn): RNN(1, 30, num_layers=5, batch_first=True)
  (fc): Linear(in_features=30, out_features=1, bias=True)
)
22
torch.Size([30, 1])
torch.Size([30, 30])
torch.Size([30])
torch.Size([30])
torch.Size([30, 30])
torch.Size([30, 30])
torch.Size([30])
torch.Size([30])
torch.Size([30, 30])
torch.Size([30, 30])
torch.Size([30])
torch.Size([30])
torch.Size([30, 30])
torch.Size([30, 30])
torch.Size([30])
torch.Size([30])
torch.Size([30, 30])
torch.Size([30, 30])
torch.Size([30])
torch.Size([30])
torch.Size([1, 30])
torch.Size([1])


In [28]:
from sklearn.metrics import mean_absolute_error

In [29]:
files = os.listdir(dpath)
files.sort()
log = dict()
mpath = '/content/drive/My Drive/LAB/output_rnn/'
model = RNN(input_dim=input_dim, hidden_dim=hidden_dim, output_dim=output_dim, num_layers=num_layers)
num_epochs = 100
optimiser = torch.optim.Adam(model.parameters(), lr=0.001)

rmse = []
maes = []

from tqdm import tqdm, trange

for t in trange(num_epochs):
    for file in files:
        try:
            df = pd.read_csv(dpath + file).set_index('Date')
            df = df[['Close']]
            scaler = MinMaxScaler(feature_range=(-1, 1))
            df['Close'] = scaler.fit_transform(df['Close'].values.reshape(-1,1))
            look_back = 30 # choose sequence length
            x_train, y_train, x_test, y_test = load_data(df, look_back)

            x_train = torch.from_numpy(x_train).type(torch.Tensor)
            x_test = torch.from_numpy(x_test).type(torch.Tensor)
            y_train = torch.from_numpy(y_train).type(torch.Tensor)
            y_test = torch.from_numpy(y_test).type(torch.Tensor)

            loss_fn = torch.nn.MSELoss()
            hist = np.zeros(num_epochs)
            seq_dim =look_back-1

            y_train_pred = model(x_train)

            loss = loss_fn(y_train_pred, y_train)
            print(f"{file}: Epoch ", t, "MSE: ", loss.item(), end = '\r')
            hist[t] = loss.item()

            # Zero out gradient, else they will accumulate between epochs
            optimiser.zero_grad()

            # Backward pass
            loss.backward()

            # Update parameters
            optimiser.step()

            y_test_pred = model(x_test)

            # invert predictions
            if t == num_epochs - 1:
                y_train_pred = scaler.inverse_transform(y_train_pred.detach().numpy())
                y_train = scaler.inverse_transform(y_train.detach().numpy())
                y_test_pred = scaler.inverse_transform(y_test_pred.detach().numpy())
                y_test = scaler.inverse_transform(y_test.detach().numpy())


                mae = mean_absolute_error(y_test, y_test_pred)
                maes.append(mae)
                rmse.append(sqrt(mean_squared_error(y_test, y_test_pred)))

        except:
            log[file] = 'ERROR'

torch.save(model.state_dict(), mpath + 'global_rnn' + '.pth')

  0%|          | 0/100 [00:00<?, ?it/s]

  1%|          | 1/100 [01:01<1:40:57, 61.19s/it]

  2%|▏         | 2/100 [01:20<59:29, 36.42s/it]  

  3%|▎         | 3/100 [01:40<46:56, 29.03s/it]

  4%|▍         | 4/100 [01:59<40:00, 25.00s/it]

...

 96%|█████████▌| 96/100 [31:51<01:18, 19.69s/it]

 97%|█████████▋| 97/100 [32:10<00:58, 19.53s/it]

 98%|█████████▊| 98/100 [32:30<00:39, 19.75s/it]

 99%|█████████▉| 99/100 [32:50<00:19, 19.55s/it]

100%|██████████| 100/100 [33:10<00:00, 19.90s/it]


In [30]:
import json
for key, val in log.items():
    log[key] = str(val)
with open(mpath + 'log.json', 'w') as json_file:
    json.dump(log, json_file)

5555

In [31]:
import numpy as np
import random
import pandas as pd

import datetime
import math, time
import itertools
import datetime
from operator import itemgetter
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler
from math import sqrt
import torch
import torch.nn as nn
from torch.autograd import Variable

import matplotlib.pyplot as plt
import os

In [32]:
def load_data(stock, look_back):
    data_raw = stock.values # convert to numpy array
    data = []

    # create all possible sequences of length look_back
    for index in range(len(data_raw) - look_back):
        data.append(data_raw[index: index + look_back])

    data = np.array(data);
    test_set_size = int(np.round(0.2*data.shape[0]));
    train_set_size = data.shape[0] - (test_set_size);

    x_train = data[:train_set_size,:-1,:]
    y_train = data[:train_set_size,-1,:]

    x_test = data[train_set_size:,:-1]
    y_test = data[train_set_size:,-1,:]

    return [x_train, y_train, x_test, y_test]

In [33]:
def MAPE(Y_Predicted,Y_actual):
    mape = np.mean(np.abs((Y_actual - Y_Predicted)/Y_actual))*100
    return mape

In [34]:
input_dim = 1
hidden_dim = 30
num_layers = 5
output_dim = 1
model = RNN(input_dim=input_dim, hidden_dim=hidden_dim, output_dim=output_dim, num_layers=num_layers)

import matplotlib.dates as mdates

model.load_state_dict(torch.load('/content/drive/My Drive/LAB/output_rnn/global_rnn.pth'))
model.eval()

log_hist = ""

def infer(file):
    global log_hist
    dpath = "/content/drive/My Drive/LAB/data/"
    mpath = '/content/drive/My Drive/LAB/output_rnn/'
    df = pd.read_csv(dpath + file).set_index('Date')
    df.index = pd.to_datetime(df.index)
    df = df[['Close']]
    scaler = MinMaxScaler(feature_range=(-1, 1))
    df['Close'] = scaler.fit_transform(df['Close'].values.reshape(-1,1))
    look_back = 30 # choose sequence length
    x_train, y_train, x_test, y_test = load_data(df, look_back)
    x_train = torch.from_numpy(x_train).type(torch.Tensor)
    x_test = torch.from_numpy(x_test).type(torch.Tensor)
    y_train = torch.from_numpy(y_train).type(torch.Tensor)
    y_test = torch.from_numpy(y_test).type(torch.Tensor)
#     print(x_train, x_train.shape)
    with torch.no_grad():
        y_train_pred = model(x_train)
        y_test_pred = model(x_test)
    y_train_pred = scaler.inverse_transform(y_train_pred.detach().numpy())
    y_train = scaler.inverse_transform(y_train.detach().numpy())
    y_test_pred = scaler.inverse_transform(y_test_pred.detach().numpy())
    y_test = scaler.inverse_transform(y_test.detach().numpy())

    err_test, err_train = MAPE(y_test_pred, y_test), MAPE(y_train_pred, y_train)

    logln = f"{file.split()[0]} had {err_train}% train error and {err_test}% test error\n"
    # print(logln)
    log_hist += logln

    figure, axes = plt.subplots(figsize=(15, 6))
    axes.xaxis.set_major_locator(mdates.AutoDateLocator())
    axes.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d'))

    axes.plot(df[len(df)-len(y_test):].index, y_test, color = 'red', label = 'Real Stock Price')
    axes.plot(df[len(df)-len(y_test):].index, y_test_pred, color = 'blue', label = 'Predicted Stock Price')
    #axes.xticks(np.arange(0,394,50))
    plt.title(f'{file} Stock Price Prediction')
    plt.xlabel('Time')
    plt.ylabel('Stock Price')
    plt.legend()
    plt.savefig('/content/drive/My Drive/LAB/output_rnn/' + file.split('.')[0] + '.png')
    plt.show()

<ipython-input-34-bd4855dbb60d>:9: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('/content/drive/My Drive/LAB/output_rnn/global_rnn.pth'))


In [35]:
for file in tqdm(files):
    try:
        infer(file)
    except:
        continue

Output hidden; open in https://colab.research.google.com to view.

In [36]:
with open('/content/drive/My Drive/LAB/output_rnn/LOG.log', 'w+') as f:
    f.write(log_hist)
f.close()

In [37]:
print(rmse)
print(maes)

[55.22265169082, 482.0610211632963, 72.8552248576758, 123.85326341834921, 195.88782264985232, 432.0598916816973, 21.961959549100534, 139.22880505215147, 1025.4761698840202, 498.3777746348647, 3.8822029602435686, 72.5774640039868, 704.3301117018923, 17.982449770703084, 108.68085086044367, 131.40583011228992, 651.4195316767835, 477.7426575835572, 462.27729435372447, 55.1538309490771, 345.35464109376034, 63.00552198641997, 46.50915043535788, 1708.9223212305467, 108.93046269581801, 143.48652484641198, 3.8304555844091, 929.1936356325306, 163.79233112198813, 184.37118640123788, 116.93552076090909, 291.3116619275651, 2.4931095534214553, 13.025311404439327, 3.5660927040343595, 329.2762660365912, 85.44327202521565, 530.4057703777364, 1368.5455783422049, 35.322542474118954, 21.940527552171076, 451.02648550279173, 37.2345717053134, 249.39380411559947, 93.82632830294757, 197.706507938535, 106.98042588921349, 2039.430986819608, 21.09438584111118]
[36.177124, 313.47302, 51.957996, 87.25252, 130.078,